In [1]:
import json
from collections import defaultdict
import subprocess
import os

In [2]:

with open('./HumanEval_Py_original.jsonl', 'r') as f:
    data = f.readlines()
    original_data = [json.loads(line) for line in data]


mapping = defaultdict(list)
for d in original_data:
    id = d['task_id'].split('/')[1]
    mapping[id].append(d)
    with open(f'./Original_Files/{id}.py', 'w') as f:
        f.write(d['prompt']+'\n'+d['canonical_solution'])



In [22]:
from execution import check_correctness

In [3]:
file_name = 'GPT3.5_Output_Py_1.0'
with open(f'./Output/{file_name}.json', 'r') as f:
    data = json.load(f)


In [4]:
split_tokens = ["\n'''","\n\"\"\"'"]

In [5]:
def clear_generated_code(data):
    data = data.split('<|endoftext|>')[0]   

    for token in split_tokens:
        data = data.split(token)[0]
        
    lines = data.split('\n')
    new_data = ''
    for line in lines:
        if 'def ' in line:
            continue
        if not line.startswith(' '):
            line = '    '+line
        new_data += line+'\n'

    

    return new_data


In [6]:
def java_clear_generated_code(data):
    data = data.split('<|endoftext|>')[0]   
    data = data.split('public static void main')[0]
    left_bracket = data.count('{')
    right_bracket = data.count('}')
    if left_bracket > right_bracket:
        data += '}'*(left_bracket-right_bracket)
    return data

In [7]:
def get_Java_compilation_result(path):

    cmd = f'javac {path}'
    # print(cmd)
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()
    if err:
        return out, err.decode('utf-8')
    
    path = path.replace('.java', '')
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()

    return out, err


In [8]:
new_data = []
# for file in os.listdir('./Files'):
#     if file.endswith('.class'):
#         os.remove(f'./Files/{file}')
#     if file.endswith('.java'):
#         os.remove(f'./Files/{file}')
for d in data:
    id = d['id']

    original_id = id.split('_')[0]
    original_item = mapping[original_id][0]
    problem ={
        'task_id': id,
        'test': original_item['test'],
        'entry_point': original_item['entry_point'],
        'prompt': d['prompt']+'\n',
    }
    for i, c in enumerate(d['output']['choices']):
    # for i, c in enumerate(d['output']):
        # print(i)
        # completion = clear_generated_code(c['message']['content'])
        completion = c['message']['content'].split('<|endoftext|>')[0]
        # completion = c['text'].split('<|endoftext|>')[0]
        problem['completion_'+str(i)] = completion
        with open(f'./Files/{id}_{str(i)}.py', 'w') as f:
            f.write(original_item['prompt']+'\n'+completion)

        # result = check_correctness(original_item, completion, 3.0)
        
        # with open(f'./Files/Main.java', 'w') as f:
        #     f.write(java_clear_generated_code(completion)+'\n'+original_item['test'])
        # out, err = get_Java_compilation_result(f'./Files/Main.java')

        # for file in os.listdir('./Files'):
        #     if file.endswith('.class'):
        #         os.remove(f'./Files/{file}')
        #     if file.endswith('.java'):
        #         os.remove(f'./Files/{file}')



        # result = {}
        # if err:
        #     result['passed'] = False
        #     result['compilation_error'] = str(err)
        # else:
        #     result['passed'] = True
        #     result['compilation_error'] = ''
        #     result['output'] = out.decode('utf-8')
        #print(result)

#         problem['result_'+str(i)] = result
#     new_data.append(problem)
#         # print(problem)
# with open(f'./Output/{file_name}_result.jsonl', 'w') as f:
#     for d in new_data:
#         f.write(json.dumps(d)+'\n')